# Notebook 4: Agentic RAG Introduction

In this notebook you will learn how to classify a user search and have specialize agents called depending on the user's question. This notebook does not use the Azure AI Search index to create a full solution for you - you will get to do that in the hands on exercise.

> NOTE: This notebook is heavily inspired by the [Agent Workflow Patterns sample](https://github.com/microsoft/agent-framework/tree/main/dotnet/samples/GettingStarted/Workflows/_Foundational/04_AgentWorkflowPatterns) in the agent-framework source tree.

## Learning Objectives
- Learn about the Handoff pattern
- Implement a classifier to recognize the user's intent and route to a specialized agent
- Implement agents to respond to specific search types


### Install Required Packages

In [ ]:
#r "nuget:Microsoft.Agents.AI.Workflows, *-*"
#r "nuget:Microsoft.Agents.AI.OpenAI, *-*"
#r "nuget:Microsoft.Extensions.AI, *-*"
#r "nuget:Microsoft.Extensions.AI.OpenAI, *-*"
#r "nuget:Azure.AI.OpenAI, *-*"
#r "nuget:Azure.Core, *-*"
#r "nuget:Azure.Identity, *-*"
#r "nuget:Azure.Search.Documents, *-*"
#r "nuget:System.Linq.AsyncEnumerable, *-*"
#r "nuget:Microsoft.Extensions.Configuration, 10.0.1"
#r "nuget:Microsoft.Extensions.Configuration.Json, 10.0.1"
#r "nuget:Microsoft.Extensions.configuration.Binder, 10.0.1"
#r "nuget:Microsoft.Extensions.Configuration.EnvironmentVariables, 10.0.1"

### Setup the Module Imports

In [ ]:
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.AI;
using System.IO;
using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using OpenAI.Embeddings;
using System.Text.Json;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using System.ComponentModel;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;
using OpenAI.Chat;


### Get the needed environment variables

In [ ]:
public const string DefaultConfigFileName = "appsettings.Local.json";

public static string? FindConfigDirectory(string fileName)
{
    var directory = new DirectoryInfo(Directory.GetCurrentDirectory());

    while (directory is not null)
    {
        if (File.Exists(Path.Combine(directory.FullName, fileName)))
        {
            return directory.FullName;
        }
        directory = directory.Parent;
    }

    return null;
}

var basePath = FindConfigDirectory(DefaultConfigFileName)
            ?? throw new InvalidOperationException(
                $"Could not find {DefaultConfigFileName} in current directory or any parent directory.");

// Load configuration from appsettings.json
var configuration = new ConfigurationBuilder()
    .SetBasePath(basePath)
    .AddJsonFile("appsettings.Local.json", optional: true, reloadOnChange: true) // Optional environment-specific settings
    .AddEnvironmentVariables()
    .Build();


foreach (var kvp in configuration.AsEnumerable())
{
    if (!string.IsNullOrEmpty(kvp.Value))
    {
        Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
    }
}

var searchSettings = new 
{
    Endpoint = Environment.GetEnvironmentVariable("AZURE_SEARCH_ENDPOINT") ?? string.Empty,
    ApiKey = Environment.GetEnvironmentVariable("AZURE_SEARCH_API_KEY") ?? string.Empty,
    IndexName = Environment.GetEnvironmentVariable("AZURE_SEARCH_INDEX_NAME") ?? string.Empty
};

var openAISettings = new
{
    Endpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT") ?? string.Empty,
    ApiVersion = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_VERSION") ?? string.Empty,
    ChatDeploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME") ?? string.Empty,
    EmbeddingDeploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDING_DEPLOYMENT") ?? string.Empty,
};


Define the AI functions you'll use for testing the specific search type in this notebook.

> NOTE: these are where the actual implementation goes for doing specific search types in the hands on exercise

In [ ]:
public class SearchTools
{
    [Description("Answers yes or no questions.")]
    public string YesOrNoSearch([Description("User question to answer with yes or no")] string userQuestion)
    {
        return $"Question: {userQuestion}\nAnswer: NO";
    }

    [Description("Answers questinos that required counting.")]
    public string CountSearch([Description("User question requiring counting items")] string userQuestion)
    {
        return $"Question: {userQuestion}\nAnswer: 42";
    }

    [Description("Answers simple question that requires semantic search.")]
    public string SemanticSearch([Description("User question requiring semantic search")]string userQuestion)
    {
        return $"Question: {userQuestion}\nAnswer: They all the gray or metallic in color.";
    }
}

Create a prompt for the classifier to be able to determine how to route the user's question.

In [ ]:
var CLASSIFIER_AGENT_INSTRUCTIONS = """
You are a query classification system for an IT support ticket database. Your task is to route them to specialist search agents based on the user question.

## Database Schema
The database contains IT support tickets with these fields:
- Id: unique identifier
- Subject: ticket subject
- Body: ticket question/description
- Answer: ticket response/solution
- Type: ticket type (e.g., "Incident", "Request", "Problem")
- Queue: department name (e.g., "Human Resources", "IT", "Finance")
- Priority: "high", "medium", or "low"
- Language: ticket language
- Business_Type: business category
- Tags: categorization tags

## Types of Searches Agents can do:

**YES_NO_AGENT**: Simple yes/no questions
   - Keywords: "is", "are", "can", "does", "do", "will", "should"
   - Examples:
     - "Is my account locked?"
     - "Can I access the VPN?"
     - "Does the printer support color printing?"
     - "Will my password expire soon?"

**COUNT_AGENT**: Questions requiring counting items
   - Keywords: "how many", "number of", "count of", "total"
    - Examples:
      - "How many devices are connected to the network?"
      - "What is the total number of open tickets?"
      - "Count of users with admin access"

**SEMANTIC_SEARCH_AGENT**: Queries looking for similar issues or solutions based on meaning
   - Keywords: "how to", "why", "what causes", "solve", "fix", "issue with", "problem with"
   - Examples:
     - "How do I reset my password?"
     - "Why is my VPN not connecting?"
     - "Issues with email synchronization"
     - "Laptop won't turn on"

**NO_SEARCH_NEEDED**: Queries that do not require any search
   - Examples:
      - "Hello I need assistance."
      - "Thank you for your help."
      - "Goodbye."
      - "No further questions."

Consider:
1. The primary intent - what is the user trying to achieve?
2. Whether they need specific data points (analytical) vs similar content (semantic)
3. Whether filters/constraints are present
4. The type of expected response (number, list, specific tickets, similar issues)
5. If no search is needed, respond with NO_SEARCH_NEEDED.

""";

Define a method that will create all the agents and connect the AI functions created earlier to their respective agents.

In [ ]:

/// <summary>
/// Create and configure the classifier and search specialist agents.
/// </summary>
/// <remarks>
/// The classifier agent is responsible for:
/// - Receiving all user input first
/// - Deciding whether to handle the request directly or hand off to a search specialist
/// - Signaling handoff by calling one of the explicit handoff tools exposed to it
///
/// Search specialist agents are invoked only when the classifier agent explicitly hands off to them.
/// After a specialist responds, control returns to the classifier agent, which then prompts
/// the user for their next message.
/// </remarks>
/// <param name="chatClient">The Azure OpenAI chat client</param>
/// <returns>Tuple of (classifierAgent, yesNoAgent, countAgent, semanticSearchAgent)</returns>
public (AIAgent classifierAgent, AIAgent yesNoAgent, AIAgent countAgent, AIAgent semanticSearchAgent)
    CreateAgents(ChatClient chatClient)
{
    // Classifier agent: Acts as the search planner and dispatcher
    var classifierAgent = chatClient.CreateAIAgent(        
                                instructions: CLASSIFIER_AGENT_INSTRUCTIONS,
                                name: "classifier_agent"
                            );

    var searchTools = new SearchTools();

    // Yes/No search specialist: Handles yes/no questions
    var yesNoAgent = chatClient.CreateAIAgent(
        instructions: "You handle yes/no questions.",
        name: "yes_no_agent",
        tools: [AIFunctionFactory.Create(searchTools.YesOrNoSearch)]
    );

    // Count search specialist: Handles counting questions
    var countAgent = chatClient.CreateAIAgent(
        instructions: "You handle questions that require counting items.",
        name: "count_agent",
        tools: [AIFunctionFactory.Create(searchTools.CountSearch)]
    );

    // Semantic search specialist: Handles semantic search questions
    var semanticSearchAgent = chatClient.CreateAIAgent(
        instructions: "You handle questions that require semantic search.",
        name: "semantic_search_agent",
        tools: [AIFunctionFactory.Create(searchTools.SemanticSearch)]
    );

    return (classifierAgent, yesNoAgent, countAgent, semanticSearchAgent);
}

Create a utility method that writes out the workflow status and messages.

In [ ]:
static async Task<List<Microsoft.Extensions.AI.ChatMessage>> RunWorkflowAsync(Workflow workflow, List<Microsoft.Extensions.AI.ChatMessage> messages)
{
    string? lastExecutorId = null;

    await using StreamingRun run = await InProcessExecution.StreamAsync(workflow, messages);
    await run.TrySendMessageAsync(new TurnToken(emitEvents: true));
    await foreach (WorkflowEvent evt in run.WatchStreamAsync())
    {
        if (evt is AgentRunUpdateEvent e)
        {
            if (e.ExecutorId != lastExecutorId)
            {
                lastExecutorId = e.ExecutorId;
                Console.WriteLine();
                Console.WriteLine(e.ExecutorId);
            }

            Console.Write(e.Update.Text);
            if (e.Update.Contents.OfType<FunctionCallContent>().FirstOrDefault() is FunctionCallContent call)
            {
                Console.WriteLine();
                Console.WriteLine($"  [Calling function '{call.Name}' with arguments: {JsonSerializer.Serialize(call.Arguments)}]");
            }
        }
        else if (evt is WorkflowOutputEvent output)
        {
            Console.WriteLine();
            return output.As<List<Microsoft.Extensions.AI.ChatMessage>>()!;
        }
    }

    return [];
}

Now all the code is defined, you get to used it.

Create a chat client and all the agents.

In [ ]:
// Initialize the Azure OpenAI chat client
var chatClient = new AzureOpenAIClient(new Uri(openAISettings.Endpoint), new DefaultAzureCredential())
                    .GetChatClient(openAISettings.ChatDeploymentName);

// Create all agents: classifier + specialists
var (classifierAgent, yesNoAgent, countAgent, semanticSearchAgent) = CreateAgents(chatClient);

Create the workflow, using the `WithHandoffs` method to use the [HandoffsWorkflowBuilder](https://github.com/microsoft/agent-framework/blob/main/dotnet/src/Microsoft.Agents.AI.Workflows/HandoffsWorkflowBuilder.cs#L13) and configure the agent targets.

In [ ]:
var workflow = AgentWorkflowBuilder.CreateHandoffBuilderWith(classifierAgent)
    .WithHandoffs(classifierAgent, [yesNoAgent, countAgent, semanticSearchAgent])
    .WithHandoffs([yesNoAgent, countAgent, semanticSearchAgent], classifierAgent)
    .Build();

For demo purposes create a list of some sample questions.

In [ ]:
var demo_questions = new List<string>() {
    "How many tickets were logged and Incidents for Human Resources and low priority?", // count search
    "Are there any issues logged for Dell XPS laptops?" // yes/no search
};

Loop through the above demo questions to see if it correctly classifies the search types.

In [ ]:
List<Microsoft.Extensions.AI.ChatMessage> messages = [];
foreach(var message in demo_questions)
{
    Console.Write("Q: ");
    messages.Add(new(ChatRole.User, message));
    messages.AddRange(await RunWorkflowAsync(workflow, messages));
}

As I've pointed out in the other notebooks, the question "How many tickets were logged and Incidents for Human Resources and low priority?" is a hard one to get the system to answer. This time the classifier correctly identified it as something the count_agent should take care of - which means we can now handle special retreival to answer it. That is a start!

Next is the hands on exercise where you get to turn these lessons from the notebooks into an Agentic RAG solution for the IT support search index we've been using.